<a href="https://colab.research.google.com/github/kty0307/Blog/blob/main/GPTs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 저축은행

## 1.1. 저축은행별 금리

- 조건
  - 가계신용대출, 저축은행별금리현황 -> 취급금액전체, 2025년 1월

In [ ]:
import requests
import json

def crawl_fsb_data(year, month):
  """
  fsb.or.kr 웹사이트에서 데이터를 크롤링하는 함수

  Args:
    year (int): 년도
    month (int): 월

  Returns:
    dict: 크롤링된 데이터 (JSON 형식)
  """

  url = "https://www.fsb.or.kr/ratloanconf_0200.jct"
  payload = {
      "_JSON_": json.dumps({
          "SORT_COLUMN": "",
          "SORT": "",
          "PRE_MONTH_MONEY": "",
          "SUBMIT_MONTH": f"{year:04d}{month:02d}"
      })
  }

  response = requests.post(url, data=payload)

  if response.status_code == 200:
    try:
      # 응답 데이터를 JSON 형식으로 파싱
      data = response.json()
      return data
    except json.JSONDecodeError:
      print("Error: Invalid JSON response")
      return None
  else:
    print(f"Error: HTTP {response.status_code}")
    return None

if __name__ == "__main__":
  year = 2025
  month = 1

  crawled_data = crawl_fsb_data(year, month)

  if crawled_data:
    # 크롤링된 데이터 출력 (예시)
    print(crawled_data)

    # TODO: 크롤링된 데이터를 파일에 저장하거나 추가 처리
  else:
    print("Failed to crawl data.")

{'REC': [{'FINAN_COMP_CODE': '0013002', 'FINAN_PROD_CODE': 'BNK2300', 'TB_SEQ': '14685', 'B_RATE3': '4.26', 'PRE_MONTH_MONEY': '1', 'B_RATE4': '4.26', 'JOIN_LOCATION': '영업점+인터넷+스마트폰', 'B_RATE1': '4.26', 'URL': 'http://www.bnksb.com/', 'B_RATE2': '4.26', 'C_RATE_AVE': '9.14', 'STATUS': 'N', 'PRODUCT_KIND': '1', 'OWNER': '경영기획부, 16449988', 'AREA': '03', 'C_RATE1': '8.76', 'B_RATE1_3': '4.26', 'C_RATE2': '9.8', 'ISUSE': '1', 'B_RATE_AVE': '4.26', 'C_RATE3': '9.65', 'C_RATE4': '7.68', 'BANK_CODE': 'fb219', 'PERMIT_YN': 'Y', 'A_RATE1_3': '12.67', 'A_RATE1': '13.02', 'A_RATE_AVE': '13.4', 'A_RATE4': '11.94', 'AREA_NAME': '부산/경남지부', 'A_RATE3': '13.91', 'A_RATE2': '14.06', 'REG_DATE': '2025-01-15 15:28:15.0', 'START_DATE': '20250115', 'C_RATE1_3': '8.41', 'CB_NAME': 'NICE', 'PRODUCT_NAME': '개인신용대출', 'SUBMIT_MONTH': '202501', 'FINAN_COMP_SUBMIT_DATE': '20250115', 'CHG_DATE': '2025-01-15 15:28:15.0', 'TEL': '16449988', 'SHEET_NUM': '11', 'SEQ': '00001', 'BANK_NAME': 'BNK'}, {'FINAN_COMP_CODE': '

In [ ]:
def create_dataframe_from_json(json_data):
  """
  JSON 데이터를 입력받아 특정 열을 가진 Pandas DataFrame으로 변환합니다.

  Args:
    json_data: JSON 형식의 문자열 또는 Python 객체 (dict, list)

  Returns:
    Pandas DataFrame: 필요한 열을 가진 DataFrame. 오류 발생 시 None 반환.
  """
  try:
    # JSON 데이터 파싱
    if isinstance(json_data, str):
      data = json.loads(json_data)
    else:  # dict or list
      data = json_data

    # 데이터 추출 및 DataFrame 생성
    df = pd.DataFrame(data['REC'])
    df = df[[
        'BANK_NAME', 'SUBMIT_MONTH', 'A_RATE1_3', 'A_RATE1', 'A_RATE2',
        'A_RATE3', 'A_RATE_AVE'
    ]]

    return df

  except (json.JSONDecodeError, KeyError, TypeError) as e:
    print(f"Error processing JSON data: {e}")
    return None

## 1.2. 금리대별 취급비중

In [4]:
import requests
import json

def fetch_fsb_loan_data(year, month):
    """
    금융위원회 대출금리 공시 페이지에서 데이터를 받아오는 함수 (파싱 없이).

    Args:
        year (int): 년도 (YYYY 형식)
        month (int): 월 (MM 형식)

    Returns:
        str: JSON 응답 문자열
        None: 오류 발생 시
    """

    url = "https://www.fsb.or.kr/ratloanconf_0300.jct"

    payload = {
        "SORT": "",
        "SUBMIT_MONTH": f"{year}{month:02}"
    }

    try:
        response = requests.post(url, data={"_JSON_": json.dumps(payload)})
        response.raise_for_status()  # HTTP 오류 발생 시 예외 발생

        return response.text  # JSON 응답 문자열 반환

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# 예시 사용
year = 2025
month = 1

data = fetch_fsb_loan_data(year, month)

if data:
    print(data)  # JSON 문자열 출력 (raw data)
    # 필요하다면 이후에 JSON 파싱 및 데이터 추출 작업 수행
    try:
        json_data = json.loads(data) #JSON 파싱
        #print(json.dumps(json_data, indent=4, ensure_ascii=False)) #JSON 데이터 예쁘게 출력
    except json.JSONDecodeError:
        print("Error: Invalid JSON response")
else:
    print("Failed to fetch data.")

{"REC":[{"FINAN_COMP_CODE":"0013002","HANDING_WEIGHT_20":"0","TB_SEQ":"4209","URL":"http://www.bnksb.com/","STATUS":"N","OWNER":"경영기획부, 16449988","AREA":"03","ISUSE":"1","BANK_CODE":"fb219","HANDING_WEIGHT_18":"0.01","PERMIT_YN":"Y","HANDING_WEIGHT_16":"34.99","HANDING_WEIGHT_14":"36.64","HANDING_WEIGHT_12":"27.67","HANDING_WEIGHT_10":"0.69","AREA_NAME":"부산/경남지부","REG_DATE":"2025-01-15 15:28:15.0","HANDING_WEIGHT_TOT":"100","SUBMIT_MONTH":"202501","FINAN_COMP_SUBMIT_DATE":"20250115","CHG_DATE":"2025-01-15 15:28:15.0","TEL":"16449988","SEQ":"00001","BANK_NAME":"BNK"},{"FINAN_COMP_CODE":"0010349","HANDING_WEIGHT_14":"28","HANDING_WEIGHT_12":"67.98","AREA_NAME":"서울지부","REG_DATE":"2025-01-15 16:52:15.0","TB_SEQ":"4216","URL":"http://www.idbsb.com/","HANDING_WEIGHT_TOT":"100","STATUS":"N","OWNER":"고객문의 1600-4902,    (공시)개인금융팀 0237051684","AREA":"01","SUBMIT_MONTH":"202501","ISUSE":"1","FINAN_COMP_SUBMIT_DATE":"20250115","CHG_DATE":"2025-01-15 16:52:15.0","TEL":"16004902","BANK_CODE":"fb002"

In [6]:
data

'{"REC":[{"FINAN_COMP_CODE":"0013002","HANDING_WEIGHT_20":"0","TB_SEQ":"4209","URL":"http://www.bnksb.com/","STATUS":"N","OWNER":"경영기획부, 16449988","AREA":"03","ISUSE":"1","BANK_CODE":"fb219","HANDING_WEIGHT_18":"0.01","PERMIT_YN":"Y","HANDING_WEIGHT_16":"34.99","HANDING_WEIGHT_14":"36.64","HANDING_WEIGHT_12":"27.67","HANDING_WEIGHT_10":"0.69","AREA_NAME":"부산/경남지부","REG_DATE":"2025-01-15 15:28:15.0","HANDING_WEIGHT_TOT":"100","SUBMIT_MONTH":"202501","FINAN_COMP_SUBMIT_DATE":"20250115","CHG_DATE":"2025-01-15 15:28:15.0","TEL":"16449988","SEQ":"00001","BANK_NAME":"BNK"},{"FINAN_COMP_CODE":"0010349","HANDING_WEIGHT_14":"28","HANDING_WEIGHT_12":"67.98","AREA_NAME":"서울지부","REG_DATE":"2025-01-15 16:52:15.0","TB_SEQ":"4216","URL":"http://www.idbsb.com/","HANDING_WEIGHT_TOT":"100","STATUS":"N","OWNER":"고객문의 1600-4902,    (공시)개인금융팀 0237051684","AREA":"01","SUBMIT_MONTH":"202501","ISUSE":"1","FINAN_COMP_SUBMIT_DATE":"20250115","CHG_DATE":"2025-01-15 16:52:15.0","TEL":"16004902","BANK_CODE":"fb002